In [35]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd

In [33]:
class Dataset(Dataset):
    def __init__(self, features_file, label_file):
        self.features = pd.read_csv(features_file)
        self.labels = pd.read_csv(label_file)

        # Compute mean and standard deviation for Z-score normalization
        self.features_mean = self.features.mean()
        self.features_std = self.features.std()
        
        self.labels_mean = self.labels.mean()
        self.labels_std = self.labels.std()

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        # Select the row
        feature_row = self.features.iloc[idx].values
        label_row = self.labels.iloc[idx].values

        # Normalize features and labels using Z-score
        normalized_features = (feature_row - self.features_mean) / self.features_std
        normalized_labels = (label_row - self.labels_mean) / self.labels_std

        # Convert to tensors
        tensor_features = torch.tensor(normalized_features, dtype=torch.float)
        tensor_labels = torch.tensor(normalized_labels, dtype=torch.float)

        return tensor_features, tensor_labels

In [17]:
len(dataset)

60761

In [34]:
dataset = Dataset('H1_AI_Dataset/Dataset1_Input.csv', 'H1_AI_Dataset/Dataset1_Labels.csv')

for features, labels in dataloader:  # This correctly unpacks each batch into features and labels
    print("Features:\n", features)
    print("Labels:\n", labels)
    print("Shape of features:", features.shape)  
    print("Shape of labels:", labels.shape)  
    break

Features:
 tensor([[-0.4869, -0.3777, -0.6607, -0.6532, -0.6401, -0.4930, -0.6558, -0.5030],
        [-0.3777, -0.6607, -0.6532, -0.6400, -0.4930, -0.6558, -0.5030, -0.3775],
        [-0.6607, -0.6532, -0.6400, -0.4930, -0.6558, -0.5030, -0.3775, -0.4131],
        [-0.6532, -0.6400, -0.4929, -0.6558, -0.5030, -0.3775, -0.4131, -0.5820],
        [-0.6400, -0.4929, -0.6558, -0.5030, -0.3774, -0.4131, -0.5820, -0.6609],
        [-0.4929, -0.6558, -0.5030, -0.3774, -0.4130, -0.5821, -0.6609, -0.5164],
        [-0.6558, -0.5029, -0.3774, -0.4130, -0.5820, -0.6610, -0.5163, -0.6170],
        [-0.5029, -0.3774, -0.4130, -0.5820, -0.6610, -0.5164, -0.6169, -0.6220],
        [-0.3774, -0.4130, -0.5820, -0.6610, -0.5163, -0.6170, -0.6220, -0.3877],
        [-0.4130, -0.5820, -0.6609, -0.5163, -0.6170, -0.6220, -0.3877, -0.5111],
        [-0.5820, -0.6609, -0.5163, -0.6170, -0.6220, -0.3877, -0.5111, -0.6613],
        [-0.6609, -0.5163, -0.6169, -0.6220, -0.3877, -0.5111, -0.6613, -0.6658],
     

In [ ]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MLP(nn.Module):
    def __init__(self, input_size=8, hidden_size=[256,256,256,256], output_size=6):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.bn1 = nn.BatchNorm1d(hidden_size[0])  # BatchNorm for the first hidden layer
        
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.bn2 = nn.BatchNorm1d(hidden_size[1])  # BatchNorm for the second hidden layer
        
        self.fc3 = nn.Linear(hidden_size[1], hidden_size[2])
        self.bn3 = nn.BatchNorm1d(hidden_size[2])  # BatchNorm for the third hidden layer
        
        self.fc4 = nn.Linear(hidden_size[2], hidden_size[3])
        self.bn4 = nn.BatchNorm1d(hidden_size[3])  # BatchNorm for the fourth hidden layer
        
        self.fc5 = nn.Linear(hidden_size[3], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)  # Apply BatchNorm after fc1
        x = F.relu(x)
        x = F.dropout(x, p=0.2)
        
        x = self.fc2(x)
        x = self.bn2(x)  # Apply BatchNorm after fc2
        x = F.relu(x)
        x = F.dropout(x, p=0.2)
        
        x = self.fc3(x)
        x = self.bn3(x)  # Apply BatchNorm after fc3
        x = F.relu(x)
        x = F.dropout(x, p=0.2)
        
        x = self.fc4(x)
        x = self.bn4(x)  # Apply BatchNorm after fc4
        x = F.relu(x)
        x = F.dropout(x, p=0.2)
        
        x = self.fc5(x)  # No BatchNorm before the final layer
        return x

In [30]:
# Instantiate the model, optimizer, and loss function
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.L1Loss()

In [31]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MLP(
  (fc1): Linear(in_features=8, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=256, out_features=256, bias=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=256, out_features=6, bias=True)
)

In [32]:
# Training loop
num_epochs = 4

for epoch in range(num_epochs):
    # Training Phase
    model.train()  # Set the model to training mode
    total_train_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        outputs = model(data)
        loss = loss_function(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Testing Phase
    model.eval()  # Set the model to evaluation mode
    total_test_loss = 0
    with torch.no_grad():  # Inference mode, gradients not needed
        for batch_idx, (data, targets) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            loss = loss_function(outputs, targets)
            total_test_loss += loss.item()

    # Print Epoch Summary
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_train_loss/len(train_loader):.4f}, Test Loss: {total_test_loss/len(test_loader):.4f}')

Epoch [1/4], Training Loss: 0.3223, Test Loss: 0.2957
Epoch [2/4], Training Loss: 0.3031, Test Loss: 0.2891
Epoch [3/4], Training Loss: 0.3001, Test Loss: 0.2911
Epoch [4/4], Training Loss: 0.2976, Test Loss: 0.2879
